# Importing and Installing libraries

In [ ]:
# Install required libs

### please update Albumentations to version>=0.3.0 for `Lambda` transform support
!pip install -U albumentations>=0.3.0 --user 
!pip install -U --pre segmentation-models --user
#This installation command is to resolve issues with respect to efficient not being found in the initila build of segmentation modle sm 
!pip install -U git+https://github.com/qubvel/segmentation_models
#uPGRADE SCKITI IMAGE
!pip install --upgrade scikit-image
#RESTART THE KERNEL POST INSTALLATION
#This is to resolve the dependency issues with skimage. 
!pip install numpy==1.17
!pip install ipdb
!pip install pandas_ml
!pip install nibabel pydicom medpy
!pip install seaborn -U

In [1]:
#RESTART THE KERNEL POST INSTALLATION of cell above
import os

#Confirmation that GPU is in working order. 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from medpy.io import load as load_segcaps
import tensorflow as tf
from sklearn.model_selection import train_test_split
import glob
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
import imageio
import albumentations as A
import random
import segmentation_models as sm
import datetime
import itertools
from sklearn.utils import class_weight
import imageio
import numpy as np
import pickle
import ipdb
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from keras.models import load_model
from scipy.spatial import distance
from collections import OrderedDict
from keras import backend as K_b
import shutil
import time
from PIL import Image

from pandas_ml import ConfusionMatrix
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import seaborn as sns
import pathlib
from medpy.io import load
from sklearn.metrics import precision_recall_fscore_support

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ec2-user/a

Segmentation Models: using `keras` framework.


In [2]:


print(tf.test.gpu_device_name())

with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

/device:GPU:0
[[22. 28.]
 [49. 64.]]


# Loading data for analysis

In [76]:
test_data_dir='/home/ec2-user/SageMaker/data/50_imgs/test/NIFTI_MR_256x256_png_256grey_lvl/t1dual_inphase'
train_data_dir='/home/ec2-user/SageMaker/data/50_imgs/train/NIFTI_MR_256x256_png_256grey_lvl/t1dual_inphase'
valid_data_dir='/home/ec2-user/SageMaker/data/50_imgs/valid/NIFTI_MR_256x256_png_256grey_lvl/t1dual_inphase'
#Destination directory
dst_dir='/home/ec2-user/SageMaker/data/250_imgs/merge/NIFTI_MR_256x256_png_256grey_lvl/t1dual_inphase'
x_dst_dir = os.path.join(dst_dir, 'images')
y_dst_dir = os.path.join(dst_dir, 'masks')
paths_merge=list(zip(glob.glob(x_dst_dir+'/*.png'),
                     glob.glob(y_dst_dir+'/*.png')))
x_test_dir = os.path.join(test_data_dir, 'images')
y_test_dir = os.path.join(test_data_dir, 'masks')

x_train_dir = os.path.join(train_data_dir, 'images')
y_train_dir = os.path.join(train_data_dir, 'masks')

x_valid_dir = os.path.join(valid_data_dir, 'images')
y_valid_dir = os.path.join(valid_data_dir, 'masks')



# Merging of test/train and validation data for running per image prediction using keras prediction generator

In [ ]:
x_merge=[x_train_dir,x_test_dir,x_valid_dir]
y_merge=[y_train_dir,y_test_dir,y_valid_dir]

for x_dir,y_dir in list(zip(x_merge,y_merge)):
    
    x_file_list=glob.glob(x_dir+'/*.png')
    y_file_list=glob.glob(os.path.join(y_dir,'*.png'))

    [shutil.copy(x_tmp,os.path.join(x_dst_dir,os.path.basename(x_tmp))) for x_tmp in x_file_list]
    [shutil.copy(y_tmp,os.path.join(y_dst_dir,os.path.basename(y_tmp))) for y_tmp in y_file_list]
    

## Resizing of all images to match 256,256 size resolution 

In [12]:
for vals in paths_merge:
    trl_img=imageio.imread(vals[0])
    trl_mask=imageio.imread(vals[1])
    
    #trl_imgs_set={vals[0]:resize_img_PIL(trl_img),vals[1]:resize_img_PIL(trl_mask)}
    #ipdb.set_trace()
    #[imageio.imwrite(k,v) for k,v in trl_imgs_set.items() if type(v) is not str]
    if trl_mask.shape!=(256,256):
        print(vals[0])
    
    

# Trouble shooting area of test dataset and model loads to ensure generators are working correctly

In [25]:
test_dataset=gen_test_dataset(dst_dir,model_gnrl_params,preprocess_input,pred_gen_var=False)

/home/ec2-user/.local/lib/python3.6/site-packages/albumentations/augmentations/transforms.py:2908: UserWarning: Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().
  "Using lambda is incompatible with multiprocessing. "


In [4]:
CLASSES = ['l_kidney','liver','r_kidney','spleen']
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)

In [ ]:
test_dataset = Dataset(
    x_dst_dir, 
    y_dst_dir, 
    classes=CLASSES,
    preprocessing=get_preprocessing(preprocess_input),
    augmentation=get_validation_augmentation(),ret_img_path=True)

In [99]:
#Local parameters for analysis
lcl_wghts_dir='/home/ec2-user/SageMaker/Masters-Thesis-UNet-repository/jupyter_notebooks/weights_history_full/cat_focal_loss/btch_sz_7/lr_0.0003/weights/t1dual_inphase_all_orgs_grey_lvl_256_optm_Adam_loss_cat_focal_loss_trn_samp_sz_250_btch_sz_7_lr_0.0003_time_2019-11-18_00000096.h5'
lcl_wghts_dir_2='/home/ec2-user/SageMaker/Masters-Thesis-UNet-repository/jupyter_notebooks/weights_history_full/cat_focal_loss/btch_sz_7/lr_0.0003/weights/t1dual_inphase_all_orgs_grey_lvl_256_optm_Adam_loss_cat_focal_loss_trn_samp_sz_250_btch_sz_7_lr_0.0003_time_2019-11-18_00000093.h5'

optimiser_tmp=keras.optimizers.Adam(0.0003)
total_loss_tmp=sm.losses.CategoricalFocalLoss()

In [102]:
start_time=time.time()
model_tmp=gen_test_model(model_gnrl_params,optimiser_tmp,total_loss_tmp,lcl_wghts_dir_2)
end_time=time.time()-start_time
print('processsing time:',end_time)

processsing time: 32.6539204120636


In [103]:
import time
start_time=time.time()
output=model_tmp.predict_generator(test_dataset,steps=len(test_dataset))
end_time=time.time()-start_time
print('processsing time:',end_time)

processsing time: 19.25001549720764


In [53]:
trl_path=os.path.join(x_dst_dir,
                      'pat_id_38_t1dual_inphase_slice_no_21_256grey_lvl_256x256.png')
trl_img=imageio.imread(trl_path)
trl_img.shape

(256, 256)

In [ ]:
img,mask,img_nm=test_dataset[1]
test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)


# Loading function calls for analysis

## Data loader and dataset functions

In [85]:
def resize_img_PIL(img:np.ndarray,shp_sp=(256,256)):
    
    if img.shape!=shp_sp:
        PIL_img=Image.fromarray(img)
        np_img_reshp=np.array(PIL_img.resize(shp_sp))
        
        return np_img_reshp
    else:
        return img

# helper function for data visualization
def visualize(fig_nm=None,figdim=(33,3.1),**images):
    """PLot images in one row."""
    n = len(images)
    print(fig_nm)
    plt.figure(figsize=figdim)
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    
    if fig_nm is not None:
        plt.savefig(fig_nm,dpi=150)
        plt.clf()
    else:
        plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x
    

# classes for data loading and preprocessing
class Dataset:
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = {'background':0,'liver':63,'r_kidney':126,'l_kidney':189,'spleen':252}
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
            ret_img_path=False
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES[cls.lower()] for cls in classes]
        self.ret_img_path=ret_img_path
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = imageio.imread(self.images_fps[i])#cv2.imread(self.images_fps[i])
        image_nm=os.path.basename(self.images_fps[i])
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image=np.expand_dims(image,axis=2)
        mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # add background if mask is not binary
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        if self.ret_img_path==True:
            return image, mask,image_nm
        else:
            return image, mask
        
    def __len__(self):
        return len(self.ids)
    
    
class CustomDataloder(keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        return batch
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)   
            
def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)

# define heavy augmentations
def get_training_augmentation(dim_sp=256):
    
    rand_int_alpha=random.uniform(0,3)
    if rand_int_alpha<=0.5:
        rand_int_sigma=random.uniform(0.1,rand_int_alpha)
    elif rand_int_alpha>=2:
        rand_int_sigma=random.uniform(rand_int_alpha/1.8,rand_int_alpha)
    else:
        rand_int_sigma=random.uniform(rand_int_alpha/1.8,rand_int_alpha)
    train_transform = [
        #A.RandomGridShuffle(p=0.4,grid=(8, 8)),
        A.ElasticTransform(p=0.9,alpha=rand_int_alpha,sigma=rand_int_sigma,border_mode=cv2.BORDER_REPLICATE), #,alpha_affine=20
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        #A.RandomSizedCrop(p=0.5),
        A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=90, shift_limit=0.1, p=0.5, border_mode=cv2.BORDER_REPLICATE),

        #A.PadIfNeeded(min_height=dim_sp, min_width=dim_sp, always_apply=True, border_mode=cv2.BORDER_REPLICATE),
        #A.RandomCrop(height=dim_sp, width=dim_sp, always_apply=True),

        A.OneOf(
            [
                A.IAASharpen(p=0.5),
                A.Blur(blur_limit=3, p=0.5)
            ],
            p=0.2,
        ),

        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)

def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        A.PadIfNeeded(256, 256)
    ]
    return A.Compose(test_transform)

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

In [86]:
def keras_flow_from_dir(dst_dir,preprocess_input,
                        target_size_var=(256, 256),batch_size_var=7):
    """Creation of template based keras image generator for batch scale prediction for efficient processing."""
    gen_test_2 =ImageDataGenerator(preprocessing_function = preprocess_input)
    
    dst_dir=dst_dir+'_keras_dataloader' if dst_dir.find('_keras_dataloader')==-1 else dst_dir
    
    dataloader=gen_test_2.flow_from_directory(dst_dir,target_size=target_size_var,
                                                batch_size=batch_size_var,
                                                class_mode=None,color_mode='grayscale',shuffle=False)
    
    return dataloader
    
#tmp_v=test_dataset.next()

In [87]:
def gen_subdir_file_lst(dir_nm:str,file_sub_str:str):
    """The purpose of this method is to generate a file list of all h5 weights sorted for completing batch prediction"""
    #ipdb.set_trace()
    final_list=[]
    for root,subdir,files in os.walk(dir_nm):
        if len(files)>0:

            file_list=glob.glob(root+file_sub_str)
            final_list=final_list+file_list
    #Sorted to ensure history part 2,3 etc are synced together 
    return sorted(final_list)

In [88]:
def get_file_info(file,add_info):
    """The purpose of this method is to pull file information from the file name presnet in the string"""
    #ipdb.set_trace()
    split_vals=file[:-14].split('_')
    split_vals.sort()
    file_dict={}
    #Iterate through additional information of set of tuples on file strings for analysis
    for param_k,param_v in add_info:

        file_dict[param_k]=[x for x in param_v if x in split_vals][0]
    
    file_dict['epoch_no']=99#int(split_vals[-1][:-3]) 

    return file_dict,split_vals

In [89]:
def gen_test_dataset(test_data_dir,model_gnrl_params,preprocess_input,ret_img_path_var):
    
    x_test_dir=os.path.join(test_data_dir,'images')
    y_test_dir=os.path.join(test_data_dir,'masks')
    
    return Dataset(x_test_dir, y_test_dir, 
                            classes=model_gnrl_params['classes'],
                           augmentation=get_validation_augmentation(),
                           preprocessing=get_preprocessing(preprocess_input),
                   ret_img_path=ret_img_path_var)

## Generate model and test model functions

In [90]:
def gen_test_model(model_gnrl_param:dict,lrn_rate,total_loss,wghts_dir:str,cls_wghts_perc=None):
    """The purpose of this method is to generate a test model from the directory for analysis"""
    
    loss_func={'cat':sm.losses.CategoricalCELoss(class_weights=cls_wghts_perc),
               'wce':sm.losses.CategoricalCELoss(class_weights=cls_wghts_perc),
           'focal':sm.losses.CategoricalFocalLoss(),
           'dice':sm.losses.DiceLoss(class_weights=cls_wghts_perc)}
    
    optim=keras.optimizers.Adam(lrn_rate)
    
    reload_model = sm.Unet(model_gnrl_param['backbone'], classes=model_gnrl_param['n_classes'],
                           activation=model_gnrl_param['activation_type'],
                           encoder_weights=None,
                           input_shape=(None, None,model_gnrl_param['input_shape_N']))
    
    reload_model.compile(optim,loss_func[total_loss],model_gnrl_param['metrics'])

    reload_model.load_weights(wghts_dir)
    
    return reload_model

## Metrics for analysis

In [91]:
def gen_test_scores(model,test_dataloader,metrics)->dict:
    """The purpose of this method is to generate a summary dictionary
    of a model test set metrics for analysis"""
    
    metric_dict={}
    
    scores = model.evaluate_generator(test_dataloader)

    metric_dict["loss"]=scores[0]
    for metric, value in zip(metrics, scores[1:]):
        metric_dict[metric.__name__]=value
        
    return metric_dict

In [92]:
def gen_per_class_dice_loss(y_true:np.ndarray,y_pred:np.ndarray,
                            channel='channel_last',
                            dice_dict=OrderedDict(left_kidney=0,liver=0,right_kidney=0,spleen=0,background=0))->dict:
    """The purpose of this method is to generate a dice loss for each organ within the logits present"""
    assert y_true.shape==y_pred.shape,'Error predicted and ground tensors incorrect shape'
    #ipdb.set_trace()
    if channel=='channel_last':
        channel_idx=2
        assert y_true.shape[2]==len(dice_dict.keys()),'dictionary and prediction labels do not match'
        tmp_dict={tup_st[0]:dice_score(y_true[:,:,i],y_pred[:,:,i]) for i,tup_st in enumerate(dice_dict.items())}
    else:
        channel_idx=0
        assert y_true.shape[0]==len(dice_dict.keys()),'dictionary and prediction labels do not match'
        tmp_dict={tup_st[0]:dice_score(y_true[i,:,:],y_pred[i,:,:]) for i,tup_st in enumerate(dice_dict.items())}
        
    return tmp_dict

In [93]:
def dice_score(y_true_arr,y_pred_arr):
    """Return single f1 score for mask image for specific class. """
    from sklearn.metrics import f1_score
    
    zero_sum_chk=np.count_nonzero(y_true_arr)+np.count_nonzero(y_pred_arr)
    
    if len(np.unique(y_true_arr))>2 or len(np.unique(y_pred_arr))>2:
        print('non binary masks')
        print(np.unique(y_true_arr))
        print(np.unique(y_pred_arr))
    
    if zero_sum_chk==0:
        return 'NaN no classes in image'
    else:
        #ipdb.set_trace()
        return f1_score(y_true_arr.astype(np.int64).flatten(),
                        y_pred_arr.astype(np.int64).flatten(),average='binary')


In [94]:
def cond_gen_dir(dst_dir_val):
    if os.path.isdir(dst_dir_val)==True:
        pass
    else:        
        try:
            os.mkdir(dst_dir_val)
        except FileNotFoundError as e:
            print('creating a nested directory',dst_dir_val)
            os.makedirs(dst_dir_val)

In [95]:
def gen_test_images(tmp_model,test_dataset,file_dict:dict,model_dir,mode,test_dataloder=None):
    """Generate testing images for analysis"""
    loss_func=file_dict['loss_type']
    lrn_rate=file_dict['learn_rate']
    epoch_no=str(file_dict['epoch_no'])
    btch_sz=str(file_dict['btch_sz'])
    dst_dir=os.path.join(model_dir,'predict_imgs',loss_func+'_btch_sz_'+btch_sz+'_lr_'+lrn_rate+'_epoch_no_'+epoch_no)
    #Generating new paths based on conditional path function for nested paths 
    #ipdb.set_trace()
    dst_dir_logits=os.path.join(dst_dir,'prob_logits')
    dst_dir_imgs=os.path.join(dst_dir,'images')
    [cond_gen_dir(x) for x in [dst_dir_logits,dst_dir_imgs]]
    #Making a directory based on initial analysis
    if mode=='per_img_visualise':
        dice_score_lst=per_img_prediction_keras(test_dataset,tmp_model,dst_dir_imgs,dst_dir_logits,file_dict)
    else:
        dice_score_lst=per_batch_img_prediction_keras(test_dataset,test_dataloder,
                                                      tmp_model,dst_dir_imgs,dst_dir_logits,file_dict)
    #Summary path and dataframe
    summary_path=os.path.join(dst_dir,'summary.csv')
    tmp_df=pd.DataFrame(dice_score_lst)
    tmp_df.to_csv(summary_path)
    
    return dice_score_lst

In [96]:
def per_batch_img_prediction_keras(test_dataset,test_dataloder,tmp_model,dst_dir_imgs,dst_dir_logits,file_dict):
    
    
    dice_score_lst=[]
    #Squeezing predicted images down to pass testing. 
    num_files=len(test_dataset)
    no_of_batches=len(test_dataloder)
    #Reseting test dataloader for each prediction to ensure consistent indexing
    test_dataloder.reset()
    img_nms_lst=test_dataloder.filenames
    pr_masks = tmp_model.predict_generator(test_dataloder,steps=no_of_batches)
    no_imgs=pr_masks.shape[0]
    #Creating logit binary mask for prediction writing logit file as well to file
    test_dataset_ids=test_dataset.ids
    
    for i in range(0,no_imgs):
        #Generating image dataset and mask 
        #ipdb.set_trace()
        img_nm=os.path.basename(img_nms_lst[i])
        dst_img_path=os.path.join(dst_dir_imgs,'predict_binary_'+img_nm)
        #Getting image mask from test dataset based on mask
        gt_mask_idx=[i for i in range(0,num_files) if test_dataset_ids[i]==img_nm][0]
        image,gt_mask,_=test_dataset[gt_mask_idx]
        #Converting softmax logit to binary logit
        pr_mask_sqz=write_logit_to_file(pr_masks[i,:,:,:],dst_dir_logits,img_nm)
        #Writing final output to file
        write_prediction_output(pr_masks[i,:,:,:],dst_dir_logits,
                                img_nm,gt_mask,dst_img_path,image,file_dict)
        
        dice_loss_per_class=gen_full_dice_row(gt_mask,pr_mask_sqz,file_dict,img_nm)
        
        dice_score_lst.append(dice_loss_per_class)  
        
    return dice_score_lst

In [97]:
def per_img_prediction_keras(test_dataset,tmp_model,dst_dir_imgs,dst_dir_logits,file_dict):
    """Per image prediction script to write all images to file prediciting on a per image basis"""
    num_files=len(test_dataset)
    
    dice_score_lst=[]
    
    for i in range(0,num_files):
        #Generating image dataset and mask 
        image,gt_mask,img_nm=test_dataset[i]
        dst_img_path=os.path.join(dst_dir_imgs,'predict_binary_'+img_nm)
        #Getting images setup for testing
        image = np.expand_dims(image, axis=0)
        #Squeezing predicted images down to pass testing. 
        pr_mask = tmp_model.predict(image)
        pr_mask_sqz_logit=pr_mask.squeeze()
        #Creating logit binary mask for prediction writing logit file as well to file
        
        write_prediction_output(pr_mask_sqz_logit,dst_dir_logits,
                                img_nm,gt_mask,dst_img_path,image,file_dict)
        
        dice_loss_per_class=gen_full_dice_row(gt_mask,pr_mask_sqz,file_dict,img_nm,file_dict)
        
        dice_score_lst.append(dice_loss_per_class)
        
    return dice_score_lst

In [98]:
def write_prediction_output(pr_mask_sqz_logit,dst_dir_logits,img_nm,
                            gt_mask,dst_img_path,image,file_dict):
    
    pr_mask_sqz=write_logit_to_file(pr_mask_sqz_logit,dst_dir_logits,img_nm)
    
    
    
    if file_dict['epoch_no']>96:
    #Writing line of predicted images to file for analysis and verification. 
        visualize(dst_img_path,
            image=denormalize(image.squeeze()),
            gt_mask_l_kidney=gt_mask[:,:,0],
            pr_mask_l_kidney=pr_mask_sqz[:,:,0],
            gt_mask_liver=gt_mask[:,:,1],
            pr_mask_liver=pr_mask_sqz[:,:,1],
            gt_mask_r_kidney=gt_mask[:,:,2],
            pr_mask_r_kidney=pr_mask_sqz[:,:,2],
            gt_mask_spleen=gt_mask[:,:,3],
            pr_mask_spleen=pr_mask_sqz[:,:,3],
            gt_mask_background=gt_mask[:,:,4],
            pr_mask_background=pr_mask_sqz[:,:,4],
        )  

In [99]:
def gen_full_dice_row(gt_mask:np.ndarray,pr_mask_sqz:np.ndarray,file_dict,img_nm)->dict:
            #Generate dice loss per image
    dice_loss_per_class=gen_per_class_dice_loss(gt_mask,pr_mask_sqz)
    dice_loss_per_class['file_nm']=img_nm
    dice_loss_per_class['loss_func']=file_dict['loss_type']
    dice_loss_per_class['btch_sz']=float(file_dict['btch_sz'])
    dice_loss_per_class['learn_rate']=float(file_dict['learn_rate'])
    dice_loss_per_class['epoch_no']=float(file_dict['epoch_no'])
    
    return dice_loss_per_class

In [100]:
def logit_binarize(logit_arr):
    """The purpose of this method is to perform softmax binarisation of logit array """
    return np.where(logit_arr.max(axis=2,keepdims=True) == logit_arr,1,0).astype(np.float64)
    
def write_logit_to_file(pr_mask_sqz_logit,dst_dir_logits,img_nm):
    
    pr_mask_sqz=logit_binarize(pr_mask_sqz_logit)
    dst_logit_path=os.path.join(dst_dir_logits,'predict_logit_'+os.path.splitext(img_nm)[0])
    np.save(dst_logit_path,pr_mask_sqz_logit)
    return pr_mask_sqz

In [101]:
def get_test_set_df(model_dir,test_data_dir,
                    model_gnrl_param_dict,add_info,mode='Test',
                    cls_wghts_perc_var=np.array([0.03987201, 0.36867433, 0.35872208, 0.2314718 , 0.00125978])):
    """The purpose of this method is to generate a """
    #Generating assertion 
    assert mode.lower() in ['test','batch_visualise','per_img_visualise'],'incorrect mode selection'
    #ipdb.set_trace()
    #Generating list of paths to models weights for analysis
    model_weights_dir=gen_subdir_file_lst(model_dir,'/*.h5')
    #Model weights directory
    
    #model_weights_dir=[x for x in model_weights_dir if x.find('focal')==-1]
    model_weights_dir=[x for x in model_weights_dir if x.find('wce')==-1]
    model_weights_dir=[x for x in model_weights_dir if x.find('cat_ce')==-1]
    tmp_dice_lst_2=['dice','0.1','0.0003']
    model_weights_dir=[x for x in model_weights_dir if len([y for y in tmp_dice_lst_2 if x.find(y)!=-1])<2]
    #ipdb.set_trace()
    #Generating dataset for analysis
    preprocess_input = sm.get_preprocessing(model_gnrl_param_dict['backbone'])
    test_dataset=gen_test_dataset(test_data_dir,model_gnrl_params,preprocess_input,ret_img_path_var=True)
    if mode.lower()=='batch_visualise':
        test_dataloader = keras_flow_from_dir(test_data_dir,preprocess_input)
    elif mode.lower()=='test':
        test_dataset=gen_test_dataset(test_data_dir,model_gnrl_params,preprocess_input,ret_img_path_var=False)
        test_dataloader = CustomDataloder(test_dataset, batch_size=1, shuffle=False)
    #ipdb.set_trace()
    #Final json list to return for analysis
    final_lst=[]
    for fl_path in model_weights_dir:
        print(fl_path)
        #ipdb.set_trace()
        #File path dictionayr information for analysis
        file_dict,split_vals=get_file_info(fl_path,add_info)
        #ipdb.set_trace()
        #Generating specific parameters for laoding the model based on file nam
        start_model=time.time()
        #Load temporary model for analysis
        tmp_model=gen_test_model(model_gnrl_param_dict,
                                 float(file_dict['learn_rate']),
                                 file_dict['loss_type'],fl_path,cls_wghts_perc_var)
        finish_model=time.time()
        
        #Test model based on dataset and analysis
        if mode.lower()=='test':
            
            tmp_score_dict=gen_test_scores(tmp_model,test_dataloader,model_gnrl_params['metrics'])
            file_dict.update(tmp_score_dict)
            final_lst.append(file_dict)
        else:
            #ipdb.set_trace()
            start=time.time()
            tmp_lst=gen_test_images(tmp_model,test_dataset,file_dict,model_dir,mode,test_dataloader)
            finish=time.time()
            print('total time loading model:',finish_model-start_model)
            print('total time predicting images:',finish-start)
            #Merging list of dictionaries for analysis
            final_lst=final_lst+tmp_lst
            #print(pd.DataFrame.from_dict(tmp_lst[:10]))
        K_b.clear_session()
        
    return final_lst
      

# Load initial parameters for analysis

In [83]:
#Loading arguments for analysis
#'efficientnetb3'densenet121
#BATCH_SIZE = 3
CLASSES = ['l_kidney','liver','r_kidney','spleen']

activation = 'sigmoid' if len(CLASSES) == 1 else 'softmax'
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

cls_wghts_perc=np.array([0.03987201, 0.36867433, 0.35872208, 0.2314718 , 0.00125978])
#Getting keys for different analysis types
add_info=[('learn_rate',['0.0003','0.001','0.01','0.1']),
          ('samp_sz',['500','250','50']),('btch_sz',['3','7']),
          ('loss_type',['dice','focal','wce'])]

model_gnrl_params={'backbone':'resnet101','n_classes':len(CLASSES)+1,
                   'metrics':metrics,'input_shape_N':1,
                   'activation_type':activation,'classes':['l_kidney','liver','r_kidney','spleen']}
#Generating weights directory for iterating for analysis
#Directory lists 
model_dir='/home/ec2-user/SageMaker/data/unet_data_aug_modified_results/data_aug_all_param_reducd_50perc/cat_focal_loss/btch_sz_3/lr_0.001/final_epch_wghts'                    
test_data_dir='/home/ec2-user/SageMaker/data/250_imgs/merge/NIFTI_MR_256x256_png_256grey_lvl/t1dual_inphase'

preprocess_input = sm.get_preprocessing(model_gnrl_params['backbone'])

In [ ]:
trl_ls=[]
for vals in model_weights_dir:
    tmp_dict={}
    val_split=vals.split('/')
    tmp_dict['lrn_rate']=val_split[-3]
    tmp_dict['btch_sz']=val_split[-4]
    tmp_dict['loss']=val_split[-5]
    trl_ls.append(tmp_dict)
    
trl_df=pd.DataFrame(trl_ls)
#trl_df.drop_duplicates(inplace=True)
trl_df.shape

In [ ]:
model_results=get_test_set_df(model_dir,test_data_dir,
                              model_gnrl_params,add_info)

In [26]:
model_df=pd.DataFrame.from_dict(model_results)
model_df.to_csv('unet_model_test_data_summary_results_05_11_2019.csv')

In [30]:
model_df.sort_values('f1-score',ascending=False)

,btch_sz,f1-score,iou_score,learn_rate,loss,loss_type,samp_sz
16,3,0.697137,0.664423,0.001,0.016060,focal,250
15,3,0.668877,0.641736,0.0003,0.021826,focal,250
18,3,0.630676,0.611101,0.1,0.024256,focal,250
17,3,0.628789,0.600515,0.01,0.020441,focal,250
7,7,0.554375,0.510995,0.1,0.001331,cat,250
11,3,0.529640,0.525482,0.1,0.927148,dice,250
13,7,0.517893,0.472351,0.001,0.936010,dice,250
9,3,0.475711,0.419127,0.001,0.937979,dice,250
0,3,0.455430,0.393119,0.0003,0.000659,cat,250
5,7,0.446474,0.388626,0.001,0.000731,cat,250


# Visualisation of results

## Testing on single set of parameters

In [107]:
fl_path='/home/ec2-user/SageMaker/Masters-Thesis-UNet-repository/jupyter_notebooks/weights_history_full/wce_loss/btch_sz_3/lr_0.001/weights/t1dual_inphase_all_orgs_grey_lvl_256_optm_Adam_loss_wce_loss_trn_samp_sz_250_btch_sz_3_lr_0.001_time_2019-11-17_00000006.h5'

loss='focal'

#gen_test_model(model_gnrl_param:dict,lrn_rate,total_loss,wghts_dir:str,cls_wghts_perc=None)
tmp_model=gen_test_model(model_gnrl_params,
                                 0.003,
                                 loss,fl_path)

#test_dataset_visualise=gen_test_dataset(test_data_dir,model_gnrl_params,preprocess_input,)

In [108]:
tmp_model.count_params()

51599768

## Testing across all parameters please check gen_test_set_df for list filters however!

In [ ]:
model_dir='/home/ec2-user/SageMaker/data/unet_data_aug_modified_results/data_aug_all_param_reducd_25perc/cat_focal_loss/btch_sz_3/final_epch_wghts'   

tmp_subdir_lst=['50','250','500']

for file_sz in tmp_subdir_lst:
    
    model_dir_gnrl=os.path.join(model_dir,file_sz+'_imgs')
    
    model_results=get_test_set_df(model_dir_gnrl,test_data_dir,
                                  model_gnrl_params,add_info,mode='batch_visualise')

## Comparing plots of predicted images to masks to determine if kidneys predict more kidneys and vice versa

### Generating initial dataset for analysis

In [116]:
src_logit_dir='/home/ec2-user/SageMaker/data/unet_predict_logits/500_imgs/500_img/predict_imgs/focal_btch_sz_3_lr_0.0003_epoch_no_99/prob_logits'
src_mask_dir='/home/ec2-user/SageMaker/data/500_imgs/'

In [117]:
#Getting the logit files for analysis
logit_dir_fl=list(pathlib.Path(src_logit_dir).rglob('*.npy'))
#Substirng to filter for masks
sub_str_chk=['/masks/','/t1dual_inphase/']
lr_rates=['lr_0.001','lr_0.0003','lr_0.01','lr_0.1']
loss_type=['focal','dice']
from sklearn.metrics import precision_recall_fscore_support
#Class dictionaries for anlysis
org_idx={'l_kidney':0,'liver':1,'r_kidney':2,'spleen':3,'background':-1}
cls_dict = {'background':0,'liver':63,'r_kidney':126,'l_kidney':189,'spleen':252}
cls_int_inv_dict={org_idx[k]:v for k,v in cls_dict.items()}
classes=['l_kidney','liver','r_kidney','spleen']
class_values = [cls_dict[cls.lower()] for cls in classes]

#Getting mask files for analysis
mask_raw_fl=list(pathlib.Path(src_mask_dir).rglob('*.png'))
#Creating basename dictionary for file list
logit_dir_dict={os.path.splitext(os.path.basename(x))[0]:x for x in logit_dir_fl if str(x).find(loss_type[0])!=-1}


#Finding only t1dual images with masks for analysis
mask_dir_fl=[x for x in mask_raw_fl if all(str(x).find(y)!=-1 for y in sub_str_chk)]
#Creating basename file name dictionary for string matching. 
bs_nm_msk_dict_pth={os.path.splitext(os.path.basename(x))[0]:x for x in mask_dir_fl}

In [118]:
cd /home/ec2-user/SageMaker/Masters-Thesis-UNet-repository/jupyter_notebooks

/home/ec2-user/SageMaker/Masters-Thesis-UNet-repository/jupyter_notebooks


In [119]:
final_src_dst_dict={}
for k,v in logit_dir_dict.items():
    
    k_mask_str=k.replace('predict_logit_','')
    #Getting final mask directory and logit directory together to run analysis against one another
    try:
        final_src_dst_dict[v]=bs_nm_msk_dict_pth[k_mask_str]
    except KeyError as e:
        print('key not found for:',k_mask_str)

key not found for: pat_id_10_t1dual_inphase_slice_no_1_256grey_lvl_256x256


In [ ]:
final_src_dst_dict

### Generating F1 score for analysis

In [ ]:


final_lst=[]
for logit_pth,mask_file_pth in final_src_dst_dict.items():
    
    #Get arrays loaded up 
    logit_arr=np.load(logit_pth)
    logit_arr=logit_binarize(logit_arr)
    
    # read data
    mask = imageio.imread(mask_file_pth)
    mask=resize_img_PIL(mask)
    mask=gen_binary_mask(mask,class_values)
    
    for org_nm,idx in org_idx.items():
        
        if np.sum((mask[:,:,idx],logit_arr[:,:,idx]))!=0:
            #print('mask_value',np.sum(mask[:,:,idx]))
            #print('logit_value',np.sum(logit_arr[:,:,idx]))
            tmp_prec,tmp_recall,tmp_f1,tmp_support=precision_recall_fscore_support(mask[:,:,idx].flatten(),
                                                                       logit_arr[:,:,idx].flatten(),
                                                                       average='binary')
            
            tmp_tp,tmp_fp,tmp_tn,tmp_fn=gen_tp_fp_fp_fn(mask[:,:,idx].flatten(),logit_arr[:,:,idx].flatten())
            
            
            
        else:
            tmp_prec,tmp_recall,tmp_f1,tmp_support=('NaN','NaN','NaN','NaN')
            tmp_tp,tmp_fp,tmp_tn,tmp_fn=('NaN','NaN','NaN','NaN')
        #Get temporary statical dictionary     
        tmp_stat_dict=gen_pred_row(mask_file_pth,logit_pth,org_nm,
                                   tmp_prec,tmp_recall,tmp_f1,tmp_support,
                                   tmp_tp,tmp_fp,tmp_tn,tmp_fn,loss_type='focal',lr_rate=0.001,samp_sz=500)
        
        
        
        final_lst.append(tmp_stat_dict) 

final_df=pd.DataFrame(final_lst)



In [69]:
final_df.to_excel('unet_focal_loss_f1_score_per_class_500_imgs_data.xlsx')

In [55]:
import pickle
with open('/home/ec2-user/SageMaker/data/per_pat_gnrl_info/per_pat_slc_no.pickle','rb') as fb:
    per_pat_per_slc_dict=pickle.load(fb)
per_pat_per_slc_dict={int(k):int(v) for k,v in per_pat_per_slc_dict.items()}

In [57]:
def gen_perc_slc_grad(pat_no,slc_no,slc_dict):
    try:
        
        total_no_slcs=slc_dict[pat_no]
    except KeyError as e:
        ipdb.set_trace()
    return slc_no/total_no_slcs
    

In [75]:
final_df['slice_no']=final_df.patient.str.split('_',expand=True)[7]
final_df['pat_id']=final_df.patient.str.extract('(\d+)')[0]

cols_num_conv=['pat_id','false_positive','false_negative','true_positive','true_negative','pat_id','slice_no',
              'precision', 'recall','F1_score']
final_df[cols_num_conv] = final_df[cols_num_conv].apply(pd.to_numeric, errors='coerce')


final_df['perc_slice_no'] = final_df.apply(lambda x: gen_perc_slc_grad(x.pat_id,
                                                                       x.slice_no,
                                                                       per_pat_per_slc_dict), axis=1)

#slice_test df results 
final_df['perc_slice_no'] = final_df['perc_slice_no'].apply(pd.to_numeric, errors='coerce')
final_df_test=final_df[final_df.pat_id.isin([2,3,8,32,39])]

In [76]:
final_df[final_df.F1_score==0].groupby(['organ_type'])['false_negative'].sum()

organ_type
l_kidney     98507.0
liver        39721.0
r_kidney     98850.0
spleen      180607.0
Name: false_negative, dtype: float64

In [79]:
final_df_test.organ_type.unique()

array(['l_kidney', 'liver', 'r_kidney', 'spleen', 'background'],
      dtype=object)

In [80]:
pwd

'/home/ec2-user/SageMaker/Masters-Thesis-UNet-repository/jupyter_notebooks'

In [ ]:
final_df[final_df.F1_score==0].groupby(['organ_type'])['false_negative'].sum()

In [ ]:
#fig,axs=plt.subplots(figsize=(20,20))
#sns.set(font_scale=1.1)
org_str='spleen'
g=sns.jointplot(data=final_df_test[(final_df_test.organ_type==org_str)],
                x='perc_slice_no',y='F1_score',kind='kde',ylim=(0,1),xlim=(0,1))
g.savefig('u_net_focal_loss_lr_0.001_samp_sz_50_'+org_str+'_f1score_wrt_per_slice_no_t1dual.jpeg')

#### Kidney specific scripting for dice scores 

In [ ]:
kidney_mask_concat=merge_arrs(mask[:,:,0],mask[:,:,2])
kidney_logit_concat=merge_arrs(logit_arr[:,:,0],logit_arr[:,:,2])
tmp_prec,tmp_recall,tmp_f1,tmp_support=precision_recall_fscore_support(kidney_mask_concat.flatten(),
                                                                   kidney_logit_concat.flatten(),
                                                                   average='binary')
tmp_stat_dict=gen_pred_row(mask_file_pth,logit_pth,'both_kidneys',
                               tmp_prec,tmp_recall,tmp_f1,tmp_support)
final_lst.append(tmp_stat_dict)
#Right kidney predicting left kidney
tmp_prec,tmp_recall,tmp_f1,tmp_support=precision_recall_fscore_support(mask[:,:,0].flatten(),
                                                                   logit_arr[:,:,2].flatten(),
                                                                   average='binary')
tmp_stat_dict=gen_pred_row(mask_file_pth,logit_pth,'right_kidney_pred_left',
                               tmp_prec,tmp_recall,tmp_f1,tmp_support)
final_lst.append(tmp_stat_dict)

#Left kidney predicting right kidney
tmp_prec,tmp_recall,tmp_f1,tmp_support=precision_recall_fscore_support(mask[:,:,2].flatten(),
                                                                   logit_arr[:,:,0].flatten(),
                                                                   average='binary')

In [ ]:
final_df.to_csv('unet_dice_lr_0.0003_epch_69_per_organ_prec_recall_f1score.csv')

final_df[['F1_score','precision','recall']]=final_df[['F1_score','precision','recall']].apply(pd.to_numeric,
                                                                                             errors='coerce')

final_df_agg=final_df.groupby('organ_type')[['F1_score','precision','recall']].mean()
final_df_agg.columns=['Dice_score','Precision','Recall']
final_df_agg.to_csv('aggregate_unet_dice_lr_0.0003_epch_69__dice_prec_recall_table.csv')
final_df_agg

## Generating logit to actual image predictions side by side.

In [163]:
#'SegCaps_multilabels_2019-11-28_11-24-37
file_name='SegCaps_multilabels_2019-11-09_01-25-53'
src_logit_dir=os.path.join('/home/ec2-user/SageMaker/data/seg_caps_predict_logits',file_name)

src_mask_dir='/home/ec2-user/SageMaker/data/500_imgs/'
dst_path=os.path.join('/home/ec2-user/SageMaker/results_segcaps_predict_imgs',file_name)

In [164]:
#Getting the logit files for analysis
logit_dir_fl=list(pathlib.Path(src_logit_dir).rglob('*.mha'))
#Substirng to filter for masks
sub_str_chk_mask=['/masks/','/t1dual_inphase/']
#Substirng to filter for images
sub_str_chk_img=['/images/','/t1dual_inphase/']
#learning rate and model names to filter logits
lr_rates=['lr_0.001','lr_0.0003','lr_0.01','lr_0.1']
segcap_model=['SegCaps_multilabels_2019-11-09_01-25-53','SegCaps_multilabels_2019-11-27_20-02-58',
             'SegCaps_multilabels_2019-11-28_11-24-37']

#Class dictionaries for anlysis
org_idx_unet={'l_kidney':0,'liver':1,'r_kidney':2,'spleen':3,'background':4}
org_idx_segcaps={'l_kidney':2,'liver':1,'r_kidney':3,'spleen':4,'background':0}
cls_dict = {'background':0,'liver':63,'r_kidney':126,'l_kidney':189,'spleen':252}
#cls int dict defined for fdining maximum arrays
cls_int_inv_dict_unet={org_idx_unet[k]:v for k,v in cls_dict.items()}
cls_int_inv_dict_segcaps={org_idx_segcaps[k]:v for k,v in cls_dict.items()}

classes=['l_kidney','liver','r_kidney','spleen']
class_values = [cls_dict[cls.lower()] for cls in classes]

#Getting mask files for analysis
mask_raw_fl=list(pathlib.Path(src_mask_dir).rglob('*.png'))
#Creating basename dictionary for file list
logit_dir_dict={os.path.splitext(os.path.basename(x))[0]:x for x in logit_dir_fl if str(x).find(file_name)!=-1}


#Finding only t1dual images with masks for analysis
mask_dir_fl=[x for x in mask_raw_fl if all(str(x).find(y)!=-1 for y in sub_str_chk_mask)]
#Finding image substring match
img_dir_fl=[x for x in mask_raw_fl if all(str(x).find(y)!=-1 for y in sub_str_chk_img)]
#Creating basename file name dictionary for string matching. 
bs_nm_msk_dict_pth={os.path.splitext(os.path.basename(x))[0]:x for x in mask_dir_fl}
bs_nm_img_dict_pth={os.path.basename(x):x for x in img_dir_fl}

In [165]:
final_src_dst_dict={}
for k,v in logit_dir_dict.items():
    
    k_mask_str=k.replace('_prediction','')
    #Getting final mask directory and logit directory together to run analysis against one another
    try:
        final_src_dst_dict[v]=bs_nm_msk_dict_pth[k_mask_str]
    except KeyError as e:
        print('key not found for:',k_mask_str)

key not found for: pat_id_10_t1dual_inphase_slice_no_1_256grey_lvl_256x256


In [ ]:
plt.imshow(logit_arr[:,:,0])

In [ ]:
plt.hist(logit_arr[:,:,0].flatten())
plt.xlabel('probability map value')
plt.ylabel('occurences')

In [168]:
concat_arr_logit_background=None
for logit_pth,mask_file_pth in final_src_dst_dict.items():
    #Get arrays loaded up 
    logit_arr,_=load(str(logit_pth))
    
    back_arr=logit_arr[:,:,0].flatten()
    if concat_arr_logit_background is None:
        concat_arr_logit_background=back_arr
    else:
        concat_arr_logit_background=np.concatenate((concat_arr_logit_background,back_arr)) 

In [ ]:
plt.hist(concat_arr_logit_background,density=True,bins=20)
plt.xticks(np.arange(0, 1, step=0.05),rotation=45)

plt.xlabel('probability map value')
plt.ylabel('occurences')

In [ ]:
final_src_dst_dict

In [129]:
#Index slicing dictionary for visualisation. first index in tuple is logit 2ns index is mask. 
mask_logit_idx_slc_segcaps={'background':(0,4),'l_kidney':(3,0),'r_kidney':(2,2),'liver':(1,1),'spleen':(4,3)}
mask_logit_idx_slc_unet={'background':(4,4),'l_kidney':(0,0),'r_kidney':(2,2),'liver':(1,1),'spleen':(3,3)}

In [167]:
final_lst=[]
for logit_pth,mask_file_pth in final_src_dst_dict.items():
    
    
    #Get arrays loaded up 
    logit_arr,_=load(str(logit_pth))
    break
    logit_arr=logit_binarize(logit_arr)
    #logit_arr=reset_logit_int(logit_arr,cls_int_inv_dict_segcaps)
    pr_mask=np.rot90(logit_arr,3)

    #pr_mask_arr=logit_binarize(np.array(rot_img))
    
    #tmp_img_path
    img_nm=os.path.basename(mask_file_pth)
    tmp_img=imageio.imread(bs_nm_img_dict_pth[img_nm])
    
    dst_img_path=os.path.join(dst_path,'binary_predict_'+img_nm)
    
    # read data
    mask = imageio.imread(mask_file_pth)
    mask=resize_img_PIL(mask)
    gt_mask=gen_binary_mask(mask,class_values)
    
    visualize(dst_img_path,
            image=tmp_img,
            gt_mask_l_kidney=gt_mask[:,:,mask_logit_idx_slc_segcaps['l_kidney'][1]],
            pr_mask_l_kidney=pr_mask[:,:,mask_logit_idx_slc_segcaps['l_kidney'][0]],
            gt_mask_liver=gt_mask[:,:,mask_logit_idx_slc_segcaps['liver'][1]],
            pr_mask_liver=pr_mask[:,:,mask_logit_idx_slc_segcaps['liver'][0]],
            gt_mask_r_kidney=gt_mask[:,:,mask_logit_idx_slc_segcaps['r_kidney'][1]],
            pr_mask_r_kidney=pr_mask[:,:,mask_logit_idx_slc_segcaps['r_kidney'][0]],
            gt_mask_spleen=gt_mask[:,:,mask_logit_idx_slc_segcaps['spleen'][1]],
            pr_mask_spleen=pr_mask[:,:,mask_logit_idx_slc_segcaps['spleen'][0]],
            gt_mask_background=gt_mask[:,:,mask_logit_idx_slc_segcaps['background'][1]],
            pr_mask_background=pr_mask[:,:,mask_logit_idx_slc_segcaps['background'][0]],
        )  
    
    
    for org_nm,idx in org_idx_segcaps.items():
        
        if np.sum((gt_mask[:,:,idx],pr_mask[:,:,idx]))!=0:
            #print('mask_value',np.sum(mask[:,:,idx]))
            #print('logit_value',np.sum(logit_arr[:,:,idx]))
            tmp_prec,tmp_recall,tmp_f1,tmp_support=precision_recall_fscore_support(gt_mask[:,:,idx].flatten(),
                                                                       pr_mask[:,:,idx].flatten(),
                                                                       average='binary')
            
            tmp_tp,tmp_fp,tmp_tn,tmp_fn=gen_tp_fp_fp_fn(gt_mask[:,:,idx].flatten(),pr_mask[:,:,idx].flatten())
            
            
            
        else:
            tmp_prec,tmp_recall,tmp_f1,tmp_support=('NaN','NaN','NaN','NaN')
            tmp_tp,tmp_fp,tmp_tn,tmp_fn=('NaN','NaN','NaN','NaN')
        #Get temporary statical dictionary     
        tmp_stat_dict=gen_pred_row(mask_file_pth,logit_pth,org_nm,
                                   tmp_prec,tmp_recall,tmp_f1,tmp_support,
                                   tmp_tp,tmp_fp,tmp_tn,tmp_fn,loss_type='WCE',lr_rate=0.1,samp_sz=250)
        
        
        
        final_lst.append(tmp_stat_dict) 

final_df_segcaps=pd.DataFrame(final_lst)
final_df_segcaps.to_csv(os.path.join(dst_path,file_name+'df_f1score_re_prec_df.csv'))

### Confusion matrix generation

In [121]:
from sklearn.metrics import precision_recall_fscore_support
y_true_arr=None
y_pred_arr=None

for logit_pth,mask_file_pth in final_src_dst_dict.items():
    
    #Get arrays loaded up 
    logit_arr=np.load(logit_pth)
    logit_arr=logit_binarize(logit_arr)
    logit_arr=reset_logit_int(logit_arr,cls_int_inv_dict)
    logit_arr=comp_logit(logit_arr)
    y_pred_arr=concat_flat_arr(logit_arr.flatten(),y_pred_arr)
    # read data
    mask = imageio.imread(mask_file_pth)
    mask=resize_img_PIL(mask)
    y_true_arr=concat_flat_arr(mask.flatten(),y_true_arr)
#Generating temporary confusion matrix     
tmp_conf_mat=ConfusionMatrix(y_true_arr,y_pred_arr)
tmp_conf_mat_df=tmp_conf_mat.to_dataframe()
#rename analysis
tmp_conf_mat_df=tmp_conf_mat.to_dataframe()
tmp_conf_mat_df.rename({0.0:'Background',
                       63:'liver',
                       126:'r_kidney',
                       189:'l_kidney',
                       252:'spleen'},axis=0,inplace=True)
tmp_conf_mat_df.rename({0.0:'Background',
                       63:'liver',
                       126:'r_kidney',
                       189:'l_kidney',
                       252:'spleen'},axis=1,inplace=True)



In [124]:
tmp_conf_mat_df.to_csv('unet_focal_loss_lr_0.001_epch_no99_samp_sz_500_conf_mat.csv')


In [123]:
tmp_conf_mat_df

Predicted,Background,liver,r_kidney,l_kidney,spleen
Actual,,,,,
Background,25154182,45349,16888,28015,13282
liver,83929,820693,955,0,823
r_kidney,6329,803,79487,12231,0
l_kidney,4713,1,825,92801,167
spleen,39390,9204,627,953,130433


In [125]:
pwd

'/home/ec2-user/SageMaker/Masters-Thesis-UNet-repository/jupyter_notebooks'

In [27]:
def concat_flat_arr(arr:np.ndarray,concat_arr)->np.ndarray:
    """The purpose of this method is to concat an array together with an arra yor none type
    primarily this function is used as an aggregatoin function at the end of a for loop. """
    if concat_arr is None:
        return arr
    else:
        #ipdb.set_trace()
        concat_arr=np.concatenate((concat_arr,arr))
        return concat_arr
    

def reset_logit_int(logit:np.ndarray,cls_lbl_dict)->np.ndarray:
    """The purpose of this method is to compress a logit down into a single layer array for analysis """
    
    for k,v in cls_lbl_dict.items():
        logit[:,:,k]=np.where(logit[:,:,k]==1,v,0)
    
    return logit

def comp_logit(logit:np.ndarray)->np.ndarray:
    return np.amax(logit,axis=2)
        
    
def gen_pred_row(mask_file_pth:str,logit_pth:str,
                 org_nm:str,tmp_prec,tmp_recall,tmp_f1,
                 tmp_support,tmp_tp,tmp_fp,tmp_tn,tmp_fn,
                loss_type=None,lr_rate=None,samp_sz=None)->dict:
    
    final_dict={'patient':os.path.splitext(os.path.basename(mask_file_pth))[0],'samp_sz':samp_sz,
                          'loss':loss_type,
                          'learn_rate':lr_rate,
                          'organ_type':org_nm,
                          'precision':tmp_prec,'recall':tmp_recall,'F1_score':tmp_f1,
                          'support_no':tmp_support,'true_positive':tmp_tp,'false_positive':tmp_fp,
                            'true_negative':tmp_tn,'false_negative':tmp_fn}
    if loss_type is None:
        final_dict['loss']=[x for x in loss_type if str(logit_pth).find(x)!=-1][0]
    if lr_rate is None:
        final_dict['learn_rate']=[x for x in lr_rates if str(logit_pth).find(x)!=-1][0]
    
    return final_dict

def gen_tp_fp_fp_fn(y_true,y_pred):
    
    #Y true y predict true positive and negatives
    pos_y_true=(y_true==1)
    pos_y_pred=(y_pred==1)
    neg_y_true=(y_true==0)
    neg_y_pred=(y_pred==0)
    #Generating false positive values 
    true_pos=len(np.where(pos_y_true&pos_y_pred)[0])
    false_pos=len(np.where(pos_y_pred&neg_y_true)[0])
    #Generating true negatives and false negatives
    true_neg=len(np.where(neg_y_true&neg_y_pred)[0])
    false_neg=len(np.where(neg_y_pred&pos_y_true)[0])
    
    return true_pos,false_pos,true_neg,false_neg

def gen_binary_mask(mask:np.ndarray,class_values:list,reord_stack=None)->np.ndarray:
    
    # extract certain classes from mask (e.g. cars)
    masks = [(mask == v) for v in class_values]
    mask = np.stack(masks, axis=-1).astype('float')

    # add background if mask is not binary
    if mask.shape[-1] != 1:
        background = 1 - mask.sum(axis=-1, keepdims=True)
        mask = np.concatenate((mask, background), axis=-1)
    if reord_stack is None:
        
        return mask
    else:
        mask=np.transpose(mask,reord_stack)

def merge_arrs(arr1,arr2):
    return np.where(arr2>0,1,arr1)
    